# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
!pip install gluonts

    100% |████████████████████████████████| 296kB 12.8MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 53.3MB/s ta 0:00:01
    100% |████████████████████████████████| 4.8MB 10.3MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 42.5MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 41.4MB/s ta 0:00:01
    100% |████████████████████████████████| 10.4MB 5.1MB/s eta 0:00:01
    100% |████████████████████████████████| 28.4MB 1.9MB/s eta 0:00:011
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
Successfully built ujson holidays
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2
You are using pip

In [1]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [9]:
def deepar(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
#         context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [13]:
%%time
res = deepar(data="m4_monthly", seed=42, epochs=100, batches=1000)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 973443
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 1000/1000 [01:28<00:00, 11.29it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[0] Elapsed time 88.554 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.990855
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 1000/1000 [01:23<00:00, 11.97it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[1] Elapsed time 83.575 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.798002
INFO:root:Epoch[2] Learning rate is 0.001
 86%|████████▌ | 860/1000 [01:22<00:13, 10.48it/s, avg_epoch_loss=6.57]


KeyboardInterrupt: 

In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results

### DeepAR - use_feat_static==False

In [7]:
def deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
#     cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [8]:
%%time
res = deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=1000)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 13463
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 1000/1000 [01:11<00:00, 13.97it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[0] Elapsed time 71.576 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.031377
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.45it/s, avg_epoch_loss=6.9]
INFO:root:Epoch[1] Elapsed time 69.187 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.904093
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.46it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[2] Elapsed time 69.186 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.615703
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.47it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[3] Elapsed tim

INFO:root:Epoch[35] Evaluation metric 'epoch_loss'=6.145966
INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 1000/1000 [01:10<00:00, 14.15it/s, avg_epoch_loss=5.95]
INFO:root:Epoch[36] Elapsed time 70.653 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=5.946676
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 1000/1000 [01:11<00:00, 14.04it/s, avg_epoch_loss=6.09]
INFO:root:Epoch[37] Elapsed time 71.246 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.088417
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 1000/1000 [01:11<00:00, 14.01it/s, avg_epoch_loss=6.12]
INFO:root:Epoch[38] Elapsed time 71.363 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.124100
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 1000/1000 [01:15<00:00, 13.16it/s, avg_epoch_loss=5.93]
INFO:root:Epoch[39] Elapsed time 75.976 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.926819
INFO:root:Epoch[40] Learning rate is 0.001
1

INFO:root:Epoch[72] Learning rate is 0.0005
100%|██████████| 1000/1000 [01:34<00:00, 10.61it/s, avg_epoch_loss=5.88]
INFO:root:Epoch[72] Elapsed time 94.294 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.879922
INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 1000/1000 [01:42<00:00,  9.78it/s, avg_epoch_loss=5.97]
INFO:root:Epoch[73] Elapsed time 102.211 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.972504
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 1000/1000 [01:21<00:00, 12.34it/s, avg_epoch_loss=6.01]
INFO:root:Epoch[74] Elapsed time 81.031 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.011640
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 1000/1000 [01:21<00:00, 12.32it/s, avg_epoch_loss=5.88]
INFO:root:Epoch[75] Elapsed time 81.186 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.881699
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 1000/1000 [01:14<00:00, 13.37it/s, avg

{'MASE': 0.9894242,
 'MSIS': 9.79866835,
 'epochs': 100,
 'sMAPE': 0.13057258,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.11604762,
 'wQuantileLoss[0.9]': 0.06786353}
Wall time: 11h 55min 14s
